In [56]:
# Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import torch
import os

In [57]:
# ignore warnings
import warnings
warnings.filterwarnings('ignore')

In [58]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


### Getting the Data
* To begin, we'll establish the directory path to our dataset.

* The dataset we're using is sourced from Kaggle and can be accessed [here](https://www.kaggle.com/datasets/michaelarman/poemsdataset).

* This collection includes poems by various authors. Our goal is to create a program that can generate a poem based on an initial line provided by the user.

In [59]:
# Reading the data from file
path = 'Data/forms'
# opening every folder within the directory and reading each of the text files within the folder and saving it in a list
data = []
for folder in os.listdir(path):
    for file in os.listdir(os.path.join(path, folder)):
        with open(os.path.join(path, folder, file), 'r', encoding='utf-8') as f:
            data.append(f.read())

### Exploratory Data Analysis
* We'll begin by examining the first five entries in our dataset.
* Next, we'll determine the length of the dataset.
* Finally, we'll calculate the number of unique words in the dataset.
* We'll also determine the number of unique characters in the dataset.

In [60]:
# Checking the first 5 data
for i in range(5):
    print("="*50)
    print(data[i])

2 ABC of H.k. and China revised vision.
Barrels tears are wines and salts.
With a whisk on goody tails!
Wiggle maces to fix the heads.
Heads in jack on boxes are ceased.
Cry to paranoid truly bosses.
Bosses are jokers take your boys.
Studs are bogs with fire apples.
True predicates worth cases.’
Descents wash in badly bands.
Wholly sales are smart with cats.
Who got tenth honors in China?
Homage grand to play and plays!
Trim the times of hearts then cry.
Tanks in steels but voice wail.
Bossy dragged by tails that whisked.
Go very timid and love the wise.
Hands are lent but laws are ends.
Cases on courts are borrowed lands.
Length long with treads to retch!
Straps on times and watch here.
Arrays tanks but all are men.
Cross all suctions steal the ends.
Cave on minds are cages on objects.
Rouser rockets powers holes.
Confine curses to stop our wounds.
Whirl your bodies and jump on grounds.
Crouch of soldiers after kicks with flings.
Block one leg and hit the middle.
Cauchy3 know the tric

In [61]:
# Checking the length of the data
print(len(data))

6322


`This indicates the number of unique words in the dataset on which our model will be trained.`

In [62]:
# checking the number of unique words in the data
words = []
for i in data:
    words.extend(i.split())
print(len(set(words)))

138244


In [63]:
char = sorted(list(set(''.join(data))))

print("Number of unique characters in the data: ", len(char))
print("Unique characters in the data: ", char)

Number of unique characters in the data:  1054
Unique characters in the data:  ['\x07', '\t', '\n', ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '=', '>', '?', '@', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', '\\', ']', '^', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '{', '|', '}', '~', '\xa0', '¡', '£', '¤', '¦', '§', '©', '«', '\xad', '®', '°', '²', '´', '·', 'º', '»', '½', '¿', 'À', 'Á', 'Ã', 'Æ', 'Ç', 'È', 'É', 'Ê', 'Ñ', 'Ó', 'Ô', 'Ö', 'Ø', 'ß', 'à', 'á', 'â', 'ã', 'ä', 'æ', 'ç', 'è', 'é', 'ê', 'ë', 'ì', 'í', 'î', 'ï', 'ð', 'ñ', 'ò', 'ó', 'ô', 'ö', 'ø', 'ù', 'ú', 'û', 'ü', 'þ', 'ā', 'ă', 'ć', 'č', 'Đ', 'đ', 'ĩ', 'ī', 'Œ', 'œ', 'ś', 'Ş', 'ş', 'Š', 'š', 'ţ', 'Ž', 'ž', 'ơ', 'ư', '˝', '̀', '́', '̃', '̄'

### Observations
* The dataset contains 6322 poems.
* The number of unique words in the dataset is 1,38,244.
* The number of unique characters in the dataset is 1054.

In [64]:
# writing the unqiue words and characters to seperate files
with open('Data/unique_words.txt', 'w', encoding='utf-8') as f:
    f.write('\n'.join(set(words)))
with open('Data/unique_characters.txt', 'w', encoding='utf-8') as f:
    f.write('\n'.join(char))

### Creating the Encoder and Decoder
* We'll now create the encoder and decoder for our model.
* The encoder will convert the input text into a tensor.
* The decoder will convert the tensor back into text.

In [65]:
# Encoder
# using a basic dictionary to convert the characters to integers
char_to_int = {char: i for i, char in enumerate(char)}

In [66]:
# Decoder
# using a basic dictionary to convert the integers back to characters
int_to_char = {i: char for i, char in enumerate(char)}

In [67]:
# Creating a function to convert the text to tensor
def text_to_tensor(text, char_to_int):
    tensor = torch.tensor([char_to_int[char] for char in text], dtype=torch.long).to(device)
    return tensor.tolist()

In [68]:
print(text_to_tensor(data[0], char_to_int))

[21, 3, 36, 37, 38, 3, 82, 73, 3, 43, 17, 78, 17, 3, 68, 81, 71, 3, 38, 75, 76, 81, 68, 3, 85, 72, 89, 76, 86, 72, 71, 3, 89, 76, 86, 76, 82, 81, 17, 2, 37, 68, 85, 85, 72, 79, 86, 3, 87, 72, 68, 85, 86, 3, 68, 85, 72, 3, 90, 76, 81, 72, 86, 3, 68, 81, 71, 3, 86, 68, 79, 87, 86, 17, 2, 58, 76, 87, 75, 3, 68, 3, 90, 75, 76, 86, 78, 3, 82, 81, 3, 74, 82, 82, 71, 92, 3, 87, 68, 76, 79, 86, 4, 2, 58, 76, 74, 74, 79, 72, 3, 80, 68, 70, 72, 86, 3, 87, 82, 3, 73, 76, 91, 3, 87, 75, 72, 3, 75, 72, 68, 71, 86, 17, 2, 43, 72, 68, 71, 86, 3, 76, 81, 3, 77, 68, 70, 78, 3, 82, 81, 3, 69, 82, 91, 72, 86, 3, 68, 85, 72, 3, 70, 72, 68, 86, 72, 71, 17, 2, 38, 85, 92, 3, 87, 82, 3, 83, 68, 85, 68, 81, 82, 76, 71, 3, 87, 85, 88, 79, 92, 3, 69, 82, 86, 86, 72, 86, 17, 2, 37, 82, 86, 86, 72, 86, 3, 68, 85, 72, 3, 77, 82, 78, 72, 85, 86, 3, 87, 68, 78, 72, 3, 92, 82, 88, 85, 3, 69, 82, 92, 86, 17, 2, 54, 87, 88, 71, 86, 3, 68, 85, 72, 3, 69, 82, 74, 86, 3, 90, 76, 87, 75, 3, 73, 76, 85, 72, 3, 68, 83, 83, 7

In [69]:
# Creating a function to convert the tensor back to text
def tensor_to_text(tensor, int_to_char):
    return ''.join([int_to_char[i] for i in tensor])

In [70]:
print(tensor_to_text(text_to_tensor(data[0], char_to_int), int_to_char))

2 ABC of H.k. and China revised vision.
Barrels tears are wines and salts.
With a whisk on goody tails!
Wiggle maces to fix the heads.
Heads in jack on boxes are ceased.
Cry to paranoid truly bosses.
Bosses are jokers take your boys.
Studs are bogs with fire apples.
True predicates worth cases.’
Descents wash in badly bands.
Wholly sales are smart with cats.
Who got tenth honors in China?
Homage grand to play and plays!
Trim the times of hearts then cry.
Tanks in steels but voice wail.
Bossy dragged by tails that whisked.
Go very timid and love the wise.
Hands are lent but laws are ends.
Cases on courts are borrowed lands.
Length long with treads to retch!
Straps on times and watch here.
Arrays tanks but all are men.
Cross all suctions steal the ends.
Cave on minds are cages on objects.
Rouser rockets powers holes.
Confine curses to stop our wounds.
Whirl your bodies and jump on grounds.
Crouch of soldiers after kicks with flings.
Block one leg and hit the middle.
Cauchy3 know the tric

In [73]:
# converting specific characters to tensor and back to text
encoded_text = text_to_tensor(data[0], char_to_int)
decoded_text = tensor_to_text(encoded_text, int_to_char)
print("Original Text: ", data[0])

Original Text:  2 ABC of H.k. and China revised vision.
Barrels tears are wines and salts.
With a whisk on goody tails!
Wiggle maces to fix the heads.
Heads in jack on boxes are ceased.
Cry to paranoid truly bosses.
Bosses are jokers take your boys.
Studs are bogs with fire apples.
True predicates worth cases.’
Descents wash in badly bands.
Wholly sales are smart with cats.
Who got tenth honors in China?
Homage grand to play and plays!
Trim the times of hearts then cry.
Tanks in steels but voice wail.
Bossy dragged by tails that whisked.
Go very timid and love the wise.
Hands are lent but laws are ends.
Cases on courts are borrowed lands.
Length long with treads to retch!
Straps on times and watch here.
Arrays tanks but all are men.
Cross all suctions steal the ends.
Cave on minds are cages on objects.
Rouser rockets powers holes.
Confine curses to stop our wounds.
Whirl your bodies and jump on grounds.
Crouch of soldiers after kicks with flings.
Block one leg and hit the middle.
Cauch

In [74]:
print("Encoded Text: ", encoded_text)

Encoded Text:  [21, 3, 36, 37, 38, 3, 82, 73, 3, 43, 17, 78, 17, 3, 68, 81, 71, 3, 38, 75, 76, 81, 68, 3, 85, 72, 89, 76, 86, 72, 71, 3, 89, 76, 86, 76, 82, 81, 17, 2, 37, 68, 85, 85, 72, 79, 86, 3, 87, 72, 68, 85, 86, 3, 68, 85, 72, 3, 90, 76, 81, 72, 86, 3, 68, 81, 71, 3, 86, 68, 79, 87, 86, 17, 2, 58, 76, 87, 75, 3, 68, 3, 90, 75, 76, 86, 78, 3, 82, 81, 3, 74, 82, 82, 71, 92, 3, 87, 68, 76, 79, 86, 4, 2, 58, 76, 74, 74, 79, 72, 3, 80, 68, 70, 72, 86, 3, 87, 82, 3, 73, 76, 91, 3, 87, 75, 72, 3, 75, 72, 68, 71, 86, 17, 2, 43, 72, 68, 71, 86, 3, 76, 81, 3, 77, 68, 70, 78, 3, 82, 81, 3, 69, 82, 91, 72, 86, 3, 68, 85, 72, 3, 70, 72, 68, 86, 72, 71, 17, 2, 38, 85, 92, 3, 87, 82, 3, 83, 68, 85, 68, 81, 82, 76, 71, 3, 87, 85, 88, 79, 92, 3, 69, 82, 86, 86, 72, 86, 17, 2, 37, 82, 86, 86, 72, 86, 3, 68, 85, 72, 3, 77, 82, 78, 72, 85, 86, 3, 87, 68, 78, 72, 3, 92, 82, 88, 85, 3, 69, 82, 92, 86, 17, 2, 54, 87, 88, 71, 86, 3, 68, 85, 72, 3, 69, 82, 74, 86, 3, 90, 76, 87, 75, 3, 73, 76, 85, 72, 3

In [75]:
print("Decoded Text: ", decoded_text)

Decoded Text:  2 ABC of H.k. and China revised vision.
Barrels tears are wines and salts.
With a whisk on goody tails!
Wiggle maces to fix the heads.
Heads in jack on boxes are ceased.
Cry to paranoid truly bosses.
Bosses are jokers take your boys.
Studs are bogs with fire apples.
True predicates worth cases.’
Descents wash in badly bands.
Wholly sales are smart with cats.
Who got tenth honors in China?
Homage grand to play and plays!
Trim the times of hearts then cry.
Tanks in steels but voice wail.
Bossy dragged by tails that whisked.
Go very timid and love the wise.
Hands are lent but laws are ends.
Cases on courts are borrowed lands.
Length long with treads to retch!
Straps on times and watch here.
Arrays tanks but all are men.
Cross all suctions steal the ends.
Cave on minds are cages on objects.
Rouser rockets powers holes.
Confine curses to stop our wounds.
Whirl your bodies and jump on grounds.
Crouch of soldiers after kicks with flings.
Block one leg and hit the middle.
Cauchy